## Mphasis HyperGraf KYC based Customer Segmentation

This sample notebook shows you how to run HyperGraf KYC based Customer Segmentation using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription toHyperGraf Home Loan Lead Identifier. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page HyperGraf KYC based Customer Segmentation 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/mphasis-marketplace-customer-segmentation-v3'# put your ARN here

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='customer-segmentation-model'

content_type='application/zip'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:

def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [6]:
# Input file name should be "Input.zip"
# Input file should be in same folder where this jupyter notebook is present for real-time inference
file_name = 'Input.zip'

<Add code snippet that shows the payload contents>

#### C. Perform real-time inference

In [7]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    output.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [8]:
import pandas as pd 
pd.read_csv("output.csv",header=None)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7
0,NaN,customer_code,age,customer_seniority,income,gender,customer_status,Cluster_Lables
1,0.0,816978,55,80,67804.53,M,1,2
2,1.0,154073,50,187,109185.81,M,0,-1
3,2.0,739359,48,91,100030.77,F,1,2
4,3.0,482251,23,132,131938.74,M,1,5
...,...,...,...,...,...,...,...,...
89996,89995.0,570253,74,117,217883,M,0,3
89997,89996.0,570247,40,117,161311,F,1,3
89998,89997.0,570094,49,117,100031,F,0,3
89999,89998.0,425968,36,145,58167.5,F,1,5


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [9]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [20]:
# upload the batch-transform job input files to S3
# please maintaine this folder structure, put your input file in "Input.zip" file
# folder structure
#-Input.zip
#-Input
#--parameters.json
#--Input.csv
transform_input_folder = "Input.zip"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/customer-segmentation-model/Input.zip


In [21]:

#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

.....................
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 165-039-557
169.254.255.130 - - [15/Jul/2021 09:58:40] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [15/Jul/2021 09:58:40] "#033GET /execution-parameters HTTP/1.1#033" 404 -
['hdbscan', 'Demographical segmentation', [250, 25]]
hdbscan
Demographical segmentation
[250, 25]
Estimated number of clusters: 2
Estimated number of noise points: 109
169.254.255.130 - - [15/Jul/2021 09:58:40] "#033POST /invocations HTTP/1.1#033" 210 -
2021-07-15T09:58:40.090:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:8080/ (Press CT

In [22]:
import os
s3_conn = boto3.client("s3")
with open('output.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket, os.path.basename(transformer.output_path)+'/Input.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [23]:
pd.read_csv("output.csv",header=None)

,0,1,2,3,4,5,6,7
0,NaN,customer_code,age,customer_seniority,income,gender,customer_status,Cluster_Lables
1,0.0,816978,55,80,67804.53,M,1,-1
2,1.0,154073,50,187,109185.81,M,0,-1
3,2.0,739359,48,91,100030.77,F,1,1
4,3.0,482251,23,132,131938.74,M,1,1
...,...,...,...,...,...,...,...,...
1000,999.0,578931,49,116,100030.77,M,0,1
1001,1000.0,836089,28,77,104356.71,F,1,-1
1002,1001.0,1159379,25,23,52498.32,F,0,0
1003,1002.0,500836,70,128,103071.18,F,0,1


### 4. Clean-up

#### A. Delete the model

In [24]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

